# Compute the global score

The aim of this notebook is to demonstrate how to compute the global score to evaluate the performance for a given model (aka `AugmentedSimulator`). From now on, we assume all the inputs to compute the score are already available.

#### Import required packages

In [6]:
import os
import math

## Acceleration reference computation

As the acceleration of simulation is one of the most important criteria in this competition, in this section we try to explain with respect to which reference the acceleration will be computed.

#### Using Grid2op solver

- The reference in this competition is the physical solver based on Newton Raphson optimisation which is implemented in [Grid2op](https://github.com/rte-france/Grid2Op) framework. It tries to solve the power flow equations in AC power system. We first provide a function which return the corresponding computation time for an indicated number of samples.

Get the AC solver time (used as the reference) with respect to which the acceleration rate is computed.

In [11]:
from lips.metrics.power_grid.compute_solver_time_grid2op import compute_solver_time_grid2op

BENCH_CONFIG_PATH = os.path.join("configs", "benchmarks", "lips_idf_2023.ini")
grid2op_solver_time = compute_solver_time_grid2op(config_path=BENCH_CONFIG_PATH, benchmark_name="Benchmark_competition", nb_samples=int(1e5))

100%|██████████| 1000/1000 [00:20<00:00, 49.00it/s]

Time required to solve one power flow:  0.00032784996554255483
Time required to solve 100000 power flows:  32.784996554255486


#### Using Security Analysis

However, a more optimized way to compute the power flow is through the security analysis which is based on the factorization of the decomposition of a matrix. This happens only during first step of power flow computation and allows to obtain a significant acceleration in comparison to the above mentioned approach. <span style="color:red">In this competition, we use this optimized version as the reference power flow computation time to calculate the speed-ups of submissions.</span>

In [16]:
from lips.config import ConfigManager
from lips.metrics.power_grid.compute_solver_time import compute_solver_time

BENCH_CONFIG_PATH = os.path.join("configs", "benchmarks", "lips_idf_2023.ini")
config = ConfigManager(path=BENCH_CONFIG_PATH, section_name="Benchmark_competition")

sa_solver_time = compute_solver_time(nb_samples=int(1e5), config=config)

In [17]:
sa_solver_time

8.68975707024849

In [20]:
print(f"The acceleration obtained using Security Analysis is :  {(grid2op_solver_time / sa_solver_time):.2f} times")

The acceleration obtained using Security Analysis is :  3.77 times


## Input results description
Hereafter, we provide the score computation procedure for the submissions. We start by an example of metrics returned by a baseline approach on `lips_idf_2023` environment. 

In [21]:
test_metrics = {"ML":{"a_or":0.02, # MAPE90 
                      "a_ex":0.02, # MAPE90 
                      "p_or":0.02, # MAPE90 
                      "p_ex":0.02, # MAPE90 
                      "v_or":1.49, # MAE
                      "v_ex":1.28  # MAE
                },
                "Physics":{
                      "CURRENT_POS": 0.2,
                      "VOLTAGE_POS": 0.1,
                      "LOSS_POS": 31.99,
                      "DISC_LINES": 0,
                      "CHECK_LOSS": 3.06,
                      "CHECK_GC": 99.99,
                      "CHECK_LC": 98.82,
                      "CHECK_JOULE_LAW": 87.82                      
                }
               }

test_ood_metrics = {"ML":{"a_or":0.03, # MAPE90 
                          "a_ex":0.03, # MAPE90 
                          "p_or":0.03, # MAPE90 
                          "p_ex":0.03, # MAPE90 
                          "v_or":2.61, # MAE
                          "v_ex":2.27  # MAE
                    },
                    "Physics":{
                          "CURRENT_POS": 0.4, # violation percentage (%)
                          "VOLTAGE_POS": 0.1,
                          "LOSS_POS": 34.94,
                          "DISC_LINES": 0,
                          "CHECK_LOSS": 6.61,
                          "CHECK_GC": 99.99,
                          "CHECK_LC": 98.95,
                          "CHECK_JOULE_LAW": 89.59 
                    }
                   } 

speed_up = sa_solver_time / 2.65 # 2.65 is the inference time of a Neural Network based approach

We define the acceptability thresholds. Each variable is associated with 2 thresholds used to determine whether the result are great, acceptable or unacceptable and whether the result should be maximized or minimized.

In [23]:
thresholds={"a_or":(0.05,0.10,"min"),
            "a_ex":(0.05,0.10,"min"),
            "p_or":(0.05,0.10,"min"),
            "p_ex":(0.05,0.10,"min"),
            "v_or":(0.2,0.5,"min"),
            "v_ex":(0.2,0.5,"min"),
            "CURRENT_POS":(1., 5.,"min"),
            "VOLTAGE_POS":(1.,5.,"min"),
            "LOSS_POS":(1.,5.,"min"),
            "DISC_LINES":(1.,5.,"min"),
            "CHECK_LOSS":(1.,5.,"min"),
            "CHECK_GC":(0.05,0.10,"min"),
            "CHECK_LC":(0.05,0.10,"min"),
            "CHECK_JOULE_LAW":(1.,5.,"min")
           }

For instance, regarding the value obtained for the variable 'a_or'

- if it is lower than 0.05, the result is great
- if it is greater than 0.05 but lower than 0.10, the result is acceptable
- if it is greater than 0.10, the result is not acceptable

For a physical criteria `CHECK_GC` (check global conservation):

- if the violation is less than 5 percent, the result is acceptable
- if it is greater than 5 percent but lower than 10 percent, the result is acceptable
- if it is greater than 10 percent, the result is not acceptable

We also define the configuration which are the coefficients considered for each category and subcategories.

In [24]:
configuration={
    "coefficients":{"test":0.33, "test_ood":0.33, "speed_up":0.34},
    "test_ratio":{"ml": 0.6, "physics":0.4},
    "test_ood_ratio":{"ml": 0.6, "physics":0.4},
    "value_by_color":{"g":2,"o":1,"r":0},
    "max_speed_ratio_allowed":50
}

We evaluate the result accuracy performances for all variables. We denote by:

- g, a great result
- o, an acceptable result
- r, a not acceptable result

In [25]:
results_test=dict()
for subcategoryName, subcategoryVal in test_metrics.items():
    results_test[subcategoryName]=[]
    for variableName, variableError in subcategoryVal.items():
        thresholdMin,thresholdMax,evalType=thresholds[variableName]
        if evalType=="min":
            if variableError<thresholdMin:
                accuracyEval="g"
            elif thresholdMin<variableError<thresholdMax:
                accuracyEval="o"
            else:
                accuracyEval="r"
        elif evalType=="max":
            if variableError<thresholdMin:
                accuracyEval="r"
            elif thresholdMin<variableError<thresholdMax:
                accuracyEval="o"
            else:
                accuracyEval="g"

        results_test[subcategoryName].append(accuracyEval)
    
print(results_test)

{'ML': ['g', 'g', 'g', 'g', 'r', 'r'], 'Physics': ['g', 'g', 'r', 'g', 'o', 'r', 'r', 'r']}


the same for OOD dataset

In [26]:
results_test_ood=dict()
for subcategoryName, subcategoryVal in test_ood_metrics.items():
    results_test_ood[subcategoryName]=[]
    for variableName, variableError in subcategoryVal.items():
        thresholdMin,thresholdMax,evalType=thresholds[variableName]
        if evalType=="min":
            if variableError<thresholdMin:
                accuracyEval="g"
            elif thresholdMin<variableError<thresholdMax:
                accuracyEval="o"
            else:
                accuracyEval="r"
        elif evalType=="max":
            if variableError<thresholdMin:
                accuracyEval="r"
            elif thresholdMin<variableError<thresholdMax:
                accuracyEval="o"
            else:
                accuracyEval="g"

        results_test_ood[subcategoryName].append(accuracyEval)
    
print(results_test_ood)

{'ML': ['g', 'g', 'g', 'g', 'r', 'r'], 'Physics': ['g', 'g', 'r', 'g', 'r', 'r', 'r', 'r']}


In [27]:
def SpeedMetric(speedUp,speedMax):
    return max(min(math.log10(speedUp)/math.log10(speedMax),1),0)

In [28]:
coefficients = configuration["coefficients"]
test_ratio = configuration["test_ratio"]
test_ood_ratio = configuration["test_ood_ratio"]
value_by_color = configuration["value_by_color"]
max_speed_ratio_allowed = configuration["max_speed_ratio_allowed"]

### Test dataset:

- ML

In [29]:
test_ml_subscore=0

test_ml_res = sum([value_by_color[color] for color in results_test["ML"]])
test_ml_res = (test_ml_res * test_ratio["ml"]) / (len(results_test["ML"])*max(value_by_color.values()))
test_ml_subscore += test_ml_res

- Physics:

In [30]:
test_physics_res = sum([value_by_color[color] for color in results_test["Physics"]])
test_physics_res = (test_physics_res*test_ratio["physics"]) / (len(results_test["Physics"])*max(value_by_color.values()))
test_physics_subscore = test_physics_res

In [31]:
test_subscore = test_ml_subscore + test_physics_subscore

In [32]:
test_subscore

0.575

### Speed up

In [33]:
speedup_score = SpeedMetric(speedUp=speed_up,speedMax=max_speed_ratio_allowed)

In [34]:
speedup_score

0.30357320039998376

### Test OOD

- ML

In [35]:
test_ood_ml_subscore=0

test_ood_ml_res = sum([value_by_color[color] for color in results_test_ood["ML"]])
test_ood_ml_res = (test_ood_ml_res * test_ood_ratio["ml"]) / (len(results_test_ood["ML"])*max(value_by_color.values()))
test_ood_ml_subscore += test_ood_ml_res

In [36]:
test_ood_ml_subscore

0.39999999999999997

- Physics

In [37]:
test_ood_physics_res = sum([value_by_color[color] for color in results_test_ood["Physics"]])
test_ood_physics_res = (test_ood_physics_res*test_ood_ratio["physics"]) / (len(results_test_ood["Physics"])*max(value_by_color.values()))
test_ood_physics_subscore = test_ood_physics_res

In [38]:
test_ood_physics_subscore

0.15000000000000002

In [39]:
test_ood_subscore = test_ood_ml_subscore + test_ood_physics_subscore

In [40]:
test_ood_subscore

0.55

- Global Score

In [41]:
globalScore=100*(coefficients["test"]*test_subscore+coefficients["test_ood"]*test_ood_subscore+coefficients["speed_up"]*speedup_score)
print(globalScore)

47.44648881359945


## Evaluate a model

In this section, we use the scoring function (available under `utils.compute_score`) to compute the score for already trained baseline models.

<span style="color:red"><b>This section is under construction for the moment ...</b></span>

### Fully Connected approach

In [1]:
### Import required packages
import os
from lips.benchmark.powergridBenchmark import PowerGridBenchmark

#Define the required paths
BENCH_CONFIG_PATH = os.path.join("configs", "benchmarks", "lips_idf_2023.ini")
DATA_PATH = os.path.join("input_data_local", "lips_idf_2023")
TRAINED_MODELS = os.path.join("input_data_local", "trained_models")
LOG_PATH = "logs.log"

benchmark_kwargs = {"attr_x": ("prod_p", "prod_v", "load_p", "load_q"),
                    "attr_y": ("a_or", "a_ex", "p_or", "p_ex", "v_or", "v_ex"),
                    "attr_tau": ("line_status", "topo_vect"),
                    "attr_physics": None}

benchmark = PowerGridBenchmark(benchmark_path=DATA_PATH,
                               config_path=BENCH_CONFIG_PATH,
                               benchmark_name="Benchmark_competition",
                               load_data_set=True, 
                               load_ybus_as_sparse=False,
                               log_path=LOG_PATH,
                               **benchmark_kwargs)

In [66]:
# load an already trained augmented simulator
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

# Indicate the path required for corresponding augmented simulator parameters
SIM_CONFIG_PATH = os.path.join("configs", "simulators", "tf_fc.ini")

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark_competition",
                         bench_kwargs=benchmark_kwargs,
                         sim_config_path=SIM_CONFIG_PATH,
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

LOAD_PATH = os.path.join(TRAINED_MODELS, "lips_idf_2023")
tf_fc.restore(path=LOAD_PATH)

In [67]:
EVALUATION_PATH = os.path.join("input_data_local", "eval_results")
metrics = benchmark.evaluate_simulator(augmented_simulator=tf_fc,
                                       eval_batch_size=128,
                                       dataset="all",
                                       shuffle=False,
                                       save_path=EVALUATION_PATH,
                                       save_predictions=False
                                      )

In [ ]:
metrics["test"]["ML"]

In [ ]:
from utils.compute_score import compute_global_score
score = compute_global_score(metrics, benchmark.config)

In [88]:
import utils
import importlib
import utils.compute_score
importlib.reload(utils.compute_score)
from utils import compute_score

In [ ]:
compute_score.configuration

In [95]:
test_metrics = compute_score.reconstruct_metric_dict(metrics, "test")
test_ood_metrics = compute_score.reconstruct_metric_dict(metrics, "test_ood_topo")

In [97]:
test_results_disc = compute_score.discretize_results(test_metrics)
test_ood_results_disc = compute_score.discretize_results(test_ood_metrics)

### LeapNet Architecture

In [39]:
SIM_CONFIG_PATH = os.path.join("configs", "simulators", "tf_leapnet.ini")

In [43]:
import tensorflow as tf
from lips.augmented_simulators.tensorflow_models.powergrid.leap_net import LeapNet
from lips.dataset.scaler.powergrid_scaler import PowerGridScaler

leap_net = LeapNet(name="tf_leapnet",                  
                   bench_config_path=BENCH_CONFIG_PATH,
                   bench_config_name="Benchmark_competition",
                   bench_kwargs=benchmark_kwargs,
                   sim_config_path=SIM_CONFIG_PATH,
                   sim_config_name="DEFAULT",
                   scaler = PowerGridScaler,
                  )

In [44]:
import pickle
LOAD_PATH_TOPO_VECT = os.path.join(TRAINED_MODELS, "lips_idf_2023", leap_net.name)
LOAD_PATH_TOPO_VECT = os.path.join(LOAD_PATH_TOPO_VECT, "topo_vect_transformer.obj")
LOAD_PATH_TOPO_VECT = os.path.abspath(LOAD_PATH_TOPO_VECT)
file_handler = open(LOAD_PATH_TOPO_VECT, "rb")
topo_vect_transformer = pickle.load(file_handler)

In [45]:
LOAD_PATH = os.path.join(TRAINED_MODELS, "lips_idf_2023")
leap_net._topo_vect_transformer = topo_vect_transformer
leap_net.restore(path=LOAD_PATH)

In [47]:
EVALUATION_PATH = os.path.join("input_data_local", "eval_results")
leapnet_metrics = benchmark.evaluate_simulator(augmented_simulator=leap_net,
                                               eval_batch_size=100000,
                                               dataset="all",
                                               shuffle=False,
                                               save_path=EVALUATION_PATH,
                                               save_predictions=False
                                              )

In [ ]:
from utils.compute_score import compute_global_score
score = compute_global_score(leapnet_metrics, benchmark.config)

In [ ]:
leapnet_metrics["test"]["ML"]